In [2]:
import bigml.api

In [3]:
#Import the project
from bigml.api import BigML
api= BigML (project='project/5d94a52e5a213962e2000331')

In [4]:
#Create the source
source = api.create_source('fulltrain_dataframe.csv')

In [5]:
api.ok(source)

True

In [6]:
#Create the dataset
dataset_1 = api.create_dataset(source)

In [7]:
api.ok(dataset_1)

True

In [8]:
#Split the trainfull in train80/val20
train_dataset = api.create_dataset(
    dataset_1, {"name": "Train80",
                     "sample_rate": 0.8, "seed": "my seed"})
val = api.create_dataset(
    dataset_1, {"name": "Train20",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

In [9]:
api.ok(train_dataset)

True

In [10]:
#Launch the model ensemble
model = api.create_ensemble(train_dataset, {"objective_field":"SeriousDlqin2yrs"})

In [11]:
api.ok(model)

True

In [12]:
#Launch the batch prediction on the validation set
batch_prediction = api.create_batch_prediction(model, val, {
    "all_fields": True})

In [13]:
api.ok(batch_prediction)

True

In [14]:
#Download the new prediction 
api.download_batch_prediction(batch_prediction,
    filename='../handson-ml2/prediction_credit.csv')

'../handson-ml2/prediction_credit.csv'

In [15]:
#Upload the modified test
test_source = api.create_source('test_dataframe.csv')

In [16]:
api.ok(test_source)

True

In [17]:
#Create the dataset test
dataset_2 = api.create_dataset(test_source)

In [18]:
api.ok(dataset_2)

True

In [19]:
#Model ensemble on the test set
model = api.create_ensemble(dataset_1, {"objective_field":"SeriousDlqin2yrs"})

In [20]:
api.ok(model)

True

In [21]:
#Batch prediction on the test set/add column probabilities
batch_prediction = api.create_batch_prediction(model, dataset_2, {"all_fields": True, "confidence": True, "probabilities": True})

In [22]:
api.ok(batch_prediction)

True

In [23]:
#Evaluation on the test
evaluation = api.create_evaluation(model, dataset_1)
api.ok(evaluation)

True

In [24]:
 api.pprint(evaluation['object']['result']['model']['average_area_under_roc_curve'])

0.88198


In [25]:
api.download_batch_prediction(batch_prediction,
    filename='../Credit_p/prediction_credit_final.csv')

'../Credit_p/prediction_credit_final.csv'

In [26]:
# Import the new prediction and remove the columns/replace Id and Probability
import pandas as pd
df = pd.read_csv('prediction_credit_final.csv')
df.drop(df.columns[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]], axis=1, inplace=True)
df.rename(columns={'field1': 'Id', '1 probability': 'Probability'}, inplace=True)
df.to_csv('prediction_credit_final_clean.csv', index=False)

In [29]:
import kaggle

In [30]:
kaggle.api.competition_submit("prediction_credit_final_clean.csv", "test", "GiveMeSomeCredit")

100%|██████████| 1.66M/1.66M [00:05<00:00, 300kB/s] 


Successfully submitted to Give Me Some Credit